# Functor

## Outline

* Incentive for Functor

* Definition of Functor

* Functor examples

* Functor laws

In this lesson, we will learn about the Functor type classe and how you can use it.

## Incentive for Functor

Imagine you have a function `add1 :: Num a => a -> a` that adds 1 to an variable that has an instance of the Num type class. 

But your input data is of type `Maybe a`. How to solve this? Let's look at an example.

In [ ]:
add1 :: Num a => a -> a
add1 n = n + 1

var1, var2 :: Maybe Int
var1 = Just 1
var2 = Nothing

applyFuncToMaybe :: (a -> a) -> Maybe a -> Maybe a
applyFuncToMaybe f Nothing = Nothing
applyFuncToMaybe f (Just n) = Just (f n)

print $ applyFuncToMaybe add1 var1
print $ applyFuncToMaybe add1 var2

We could also have a custom data type:
```haskell
data Wrapper a = Empty | Wrapper a deriving Show
```
We solve it similar as the previous example.

In [ ]:
data Wrapper a = Empty | Wrapper a deriving Show

wrapperData :: [Wrapper Int]
wrapperData = [Wrapper 3, Empty, Wrapper 2, Wrapper 1, Empty]

applyFuncToWrapper :: (a -> a) -> Wrapper a -> Wrapper a
applyFuncToWrapper f Empty = Empty
applyFuncToWrapper f (Wrapper n) = Wrapper (f n) 

print $ map (applyFuncToWrapper add1) wrapperData

A concern we have now is that for every data type that puts a variable into a context, if we want to perform similiar operations we need to define such a helper function.

This is where the Functor type class comes to help to solve this problem.

## Definition of functor

The definition of the Functor type class is as follows:
```haskell
class Functor f where
  fmap :: (a -> b) -> f a -> f b
  (<$) :: a -> f b -> f a
  {-# MINIMAL fmap #-}
```

Let's see how the `fmap` function can help us to avoid using helper functions for transformations. 

We look at an example with the Maybe data type that has an instance of Functor.

In [ ]:
add1 :: Int -> Int
add1 n = n + 1

add1ToMaybe :: Maybe Int
add1ToMaybe n = fmap add1 n

print $ add1ToMaybe (Just 1)
print $ add1ToMaybe Nothing

maybeData :: [Maybe Int]
maybeData = [Just 3, Nothing, Just 2, Just 1, Nothing]

print $ map add1ToMaybe maybeData

Another way you can use the `fmap` function is in its infix form where the operator `<$>` is just a synonim for `fmap` in infix form.

In [ ]:
print $ add1 `fmap` (Just 1 :: Maybe Int)
print $ add1 <$> (Just 1 :: Maybe Int)

Also lists have an instance of Functor type class. Imagine that a list is just a context as Maybe, where in the case of Maybe a value might be missing and in the case of lists the are many possible values. 

For lists it turns out that `fmap` is just `map` so you could re-write the previous Wrapper print statement to:

In [ ]:
print $ fmap (applyFuncToWrapper add1) wrapperData

## Functor examples

Lets try now to use the Wrapper code example by making the Wrapper type an instance of the Functor type class.

In [ ]:
instance Functor Wrapper where
    (fmap) f Empty = Empty
    (fmap) f (Wrapper n) = Wrapper (f n)

print $ map (fmap add1) wrapperData

We get the exact same result as before. A curious reader may say that we only switched 3 lines of code for the `applyFuncForWrapper` function to another 3 lines of code for the `instance` declaration. 

But you do not have to come up with names for a transformation function every time you creare a new type, which makes it also easier to refactor code. And you also get the `<$` operator for free. 

Another benifit you have is that for Haskell types that have an instance of Functor the `<$>` operator works out of the box.

Here is an example where we use the `<$` operator, where we want to set all elements in a list or a Map to a single value.

In [ ]:
import qualified Data.Map as Map

myList = [1..5]
myMap = Map.fromList (zip ['a'..'c'] [1..3])

setAllTo3 :: Functor f => f Int -> f Int
setAllTo3 var = (<$) 3 var

print $ setAllTo3 myList
print $ setAllTo3 myMap

Lets look at another example how you can use the `<$>` operator on a Map which has also an instance of Functor. 

We will define a data structure with record syntax called Book that contains some book details. 

Then we will create a Map for 3 books and use the function `describe` on the Map to get a list of book descriptions.

In [ ]:
data Book = Book {
    price :: Int , 
    yearPublished :: Int ,
    title :: String
} deriving Show

book1 = Book {
    price = 15 ,
    yearPublished = 1997 ,
    title = "Harry Potter and the Philosopher's Stone"
}

book2 = Book {
    price = 17 ,
    yearPublished = 1998 ,
    title = "Harry Potter and the Chamber of Secrets"
}

book3 = Book {
    price = 19 ,
    yearPublished = 1999 ,
    title = "Harry Potter and the Prisoner of Azkaban"
}

books :: Map.Map Int Book
books = Map.fromList $ zip [1..3] [book1, book2, book3]

describe :: Book -> String
describe myBook = "The book " ++ title myBook ++
                   " was published in " ++ show (yearPublished myBook) ++
                   " and costs " ++ show (price myBook) ++ "$."

printDescriptions :: Int -> IO ()
printDescriptions x = if x < length descriptions
                      then do
                        print $ descriptions !! x
                        printDescriptions (x + 1)
                      else putStrLn "Finished."

descriptions :: [String]
descriptions = map snd $ Map.toList $ describe <$> books

printDescriptions 0

We see that the `<$>` operator by default applies a function to the values of the Map and not its keys.

## Functor laws

Haskell functors are more than just the functions given by their typeclasses. The typeclass also come with a set of laws.

The reason for this is that the term Functor comes from mathematics and is defined by these laws.

Functor has 2 laws. They are as follows:

- Identity:<br>`fmap id = id`

- Composition:<br>`fmap (f . g) = fmap f . fmap g`

The identity law ensures that `fmap` should only be a mapping function and should not change anything in the functor except for the value that it’s mapping.

The composition law says it should not matter whether we compose `fmap` applied functions, or we `fmap` a composed function.

In Haskell the role of these two laws is to guarantee `fmap` behaves the way it was intended and performs only a mapping operation.

Let's look how to test this on the Maybe type that has a correct implementation of Functor.

In [ ]:
fmap id (Just 1) == id (Just 1)

fmap ((+1) . (*2)) (Just 1) == (fmap (+1) . fmap (*2)) (Just 1)

Since they both return `True` we see that Functor is properly implemented for the Maybe type.

Let's create a type and an instance of Functor for it, that violates these laws.

In [ ]:
data NotOk a = NotOk a Bool deriving Eq

instance Functor NotOk where
  fmap f (NotOk val check) = NotOk (f val) check'
    where check' = not check

var :: NotOk Int
var = NotOk 1 True

print $ fmap id var == id var
print $ fmap ((+1) . (*2)) var == (fmap (+1) . fmap (*2)) var

Haskell does not enforce this rules and the above code still compiles. So they are more like guidelines.

Still you should follow them because:
- you can better reason about what your code is doing
- you can make use of other functions that work with the Functor type class

## Recap

In this lesson we've discussed:

- the motivation for introducing the Functor type class

- definition of the Functor type class

- examples that show how to use the Functor type class

- Functor laws and examples